# Project:      Price recommander system for Mobile industry

### Use Case: Mobile industry and websites for mobile prices (e.g. whatmobile.com,pk here in Pakistan)

# Data integration, Visualization and ETL
I used 'train.csv' file where i have my dataset for training

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3c4c594765474e10870775abb04cbf2f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QoPFoBkCOcy5_z9sCxxIlro_taDDVczipnPs_wXceqeF',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3c4c594765474e10870775abb04cbf2f.get_object(Bucket='defaultikram-donotdelete-pr-ht8dbkrt3n4iv5',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

print(type(df_data_1))

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200423192340-0003
KERNEL_ID = 76918be7-d421-43f7-8d02-bcf67bb57a07
<class 'pandas.core.frame.DataFrame'>


In [2]:
from pyspark.sql.types import *


mySchema = StructType([ StructField("battery_power", IntegerType(), True)\

                       ,StructField("blue", IntegerType(), True)\

                       ,StructField("clock_speed", DoubleType(), True)\

                       ,StructField("dual_sim", IntegerType(), True)\

                       ,StructField("fc", IntegerType(), True)\

                       ,StructField("four_g", IntegerType(), True)\

                       ,StructField("int_memory", IntegerType(), True)\

                       ,StructField("m_dep", DoubleType(), True)\

                       ,StructField("mobile_wt", IntegerType(), True)\

                       ,StructField("n_cores", IntegerType(), True)\

                       ,StructField("pc", IntegerType(), True)\

                       ,StructField("px_height", IntegerType(), True)\

                       ,StructField("px_width", IntegerType(), True)\

                       ,StructField("ram", IntegerType(), True)\

                       ,StructField("sc_h", IntegerType(), True)\

                       ,StructField("sc_w", IntegerType(), True)\

                       ,StructField("talk_time", IntegerType(), True)\

                       ,StructField("three_g", IntegerType(), True)\

                       ,StructField("touch_screen", IntegerType(), True)\

                       ,StructField("wifi", IntegerType(), True)\

                       ,StructField("price_range", IntegerType(), True)])

In [3]:

df = spark.createDataFrame(df_data_1,schema=mySchema)
df.show()


+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [4]:
df.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- price_range: integer (nullable = true)



In [5]:
df.createOrReplaceTempView('Dataset')

# Initial Data Exploration
Now, I am going to implement some simple functions that i learned earlier so that i can have a view of my dataset. I will find min, mix, standard deviation and some other properties of my data columns and will explore them on different columns. I have imported data using pandas dataframe but now converted it to spark and created a view so that i can run quries on it.

In [6]:
"""
    function to find minimum of a column
"""
def findMin(columnName):
    return spark.sql("SELECT min(battery_power) as minVal from Dataset").first().minVal

In [7]:
"""
    function to find maximum of a column
"""
def findMax(columnName):
    return spark.sql('select max('+columnName+') as maxVal from Dataset').first().maxVal

In [8]:
"""
    function to find mean of a column
"""
def findMean(columnName):
    return spark.sql('select avg('+columnName+') as meanVal from Dataset').first().meanVal

In [9]:
"""
    function to find starndard deviation of a column
"""
def findstdDev(columnName):
    return spark.sql('select stddev('+columnName+') as stddev from Dataset').first().stddev

In [10]:
"""
    function to find skewness for a column
"""
def findSkew(columnName):    
    return spark.sql("""
SELECT 
    (
        1/count(*)
    ) *
    SUM (
        POWER("""+columnName+"""-%s,3)/POWER(%s,3)
    )
    as skewVal
 from Dataset
                    """ %(findMean(columnName),findstdDev(columnName))).first().skewVal


In [11]:
minVal = findMin('battery_power')
print(minVal)

501


In [12]:
findSkew('battery_power')

0.031850640034327575

In [13]:
"""
    function to find kurtosis for specific column
"""
def kurtosis(columnName):    
        return spark.sql("""
SELECT 
    (
        1/count(*)
    ) *
    SUM (
        POWER("""+columnName+"""-%s,4)/POWER(%s,4)
    )
as kut
from Dataset
                    """ %(findMean(columnName),findstdDev(columnName))).first().kut


In [14]:
kurtosis('battery_power')

1.7741403624192582

In [15]:
def correlationinMobileWeightandBatteryPower():
    return spark.sql("SELECT corr(battery_power,mobile_wt) as corrVal from Dataset").first().corrVal

In [16]:
correlationinMobileWeightandBatteryPower()

0.0018443762145708168

# Feature Creation
As i have checked and visualized data above. I would have loved to plot it using matplotlib.pyplot but as data has 21 dimensions so it is impossible to plot hyperdimension here. I have visualized data using different parameters. As i pbtained dataset from Kaggle and checked data. there was no need of big cleaning because data was not redundant or all the columns have values. There is null value in any cell. Now, it is time to create features and define model using sparkml

In [17]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=['battery_power','blue','clock_speed','dual_sim', 'fc','four_g',
                                             'int_memory','m_dep','mobile_wt','n_cores', 'pc','px_height','px_width',
                                             'ram','sc_h','sc_w','talk_time','three_g','touch_screen','wifi'], outputCol='features')

normalizer = Normalizer(inputCol='features', outputCol='features_normalized', p=1.0)

# Creating ML Pipeline

In [18]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages =[vectorAssembler, normalizer])

model = pipeline.fit(df)

new_df = model.transform(df)

new_df.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------------------+--------------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|            features| features_normalized|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------------------+--------------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7| 

# Deleting useless columns
this now has become a big dataframe and i don't want that. I have created features and normalized. I have performed all of them using ML pipeline availabel in Apache spark. Now its time to delete that columns that i don't need
Let's do it.

In [19]:
df2 = new_df.drop('battery_power').drop('blue').drop('clock_speed').drop('dual_sim').drop('fc')\
.drop('four_g').drop('int_memory').drop('m_dep').drop('mobile_wt').drop('n_cores').drop('pc')\
.drop('px_height').drop('px_width').drop('ram').drop('sc_h').drop('sc_w').drop('talk_time')\
.drop('three_g').drop('touch_screen').drop('wifi')

In [20]:
df2.show()

+-----------+--------------------+--------------------+
|price_range|            features| features_normalized|
+-----------+--------------------+--------------------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|
|          3|[1821.0,0.0,1.7,0...|[0.27360829389226...|
|          0|[1954.0,0.0,0.5,1...|[0.42857456188449...|
|          0|[1445.0,1.0,0.5,0...|[0.35633260998224...|
|          0|[509.0,1.0,0.6,1....|[0.14327131477467...|
|          3|[769.0,1.0,2.9,1....|[0.12704444077317...|
|          3|[1520.0,1.0,2.2,0...|[0.22396746577865...|
|          1|[1815.0,0.0,2.8,0...|[0.37105941039375...|
|          2|[803.0,1.0,2.1,0....|[0.14541569330508...|
|          0|[1866.0,0.0,0.5,0...|[0.53989931138

# Machine Learning Algorithms

## 1. Decision Tree Classifier

In [21]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(featuresCol="features_normalized", labelCol="price_range")

model = classifier.fit(df2)

model = model.transform(df2)

model.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|  [10.0,8.0,0.0,0.0]|[0.55555555555555...|       0.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[0.0,7.0,40.0,365.0]|[0.0,0.0169902912...|       3.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[45.0,165.0,40.0,...|[0.18,0.66,0.16,0.0]|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[45.0,165.0,40.0,...|[0.18,0.66,0.16,0.0]|       1.0|
|          3|[1821.

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model) 

0.8115

In [23]:
df2.show()

+-----------+--------------------+--------------------+
|price_range|            features| features_normalized|
+-----------+--------------------+--------------------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|
|          3|[1821.0,0.0,1.7,0...|[0.27360829389226...|
|          0|[1954.0,0.0,0.5,1...|[0.42857456188449...|
|          0|[1445.0,1.0,0.5,0...|[0.35633260998224...|
|          0|[509.0,1.0,0.6,1....|[0.14327131477467...|
|          3|[769.0,1.0,2.9,1....|[0.12704444077317...|
|          3|[1520.0,1.0,2.2,0...|[0.22396746577865...|
|          1|[1815.0,0.0,2.8,0...|[0.37105941039375...|
|          2|[803.0,1.0,2.1,0....|[0.14541569330508...|
|          0|[1866.0,0.0,0.5,0...|[0.53989931138

## 2. Random Forest Classifier

In [24]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features_normalized", labelCol="price_range")

model1 = classifier.fit(df2)

model1 = model1.transform(df2)

model1.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|[1.98681159578887...|[0.09934057978944...|       2.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[0.96935896239778...|[0.04846794811988...|       3.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[0.40987511496948...|[0.02049375574847...|       3.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[0.32296565751008...|[0.01614828287550...|       3.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[2.30971701616857...|[0.11548585080842...|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[2.72999534444620...|[0.13649976722231...|       1.0|
|          3|[1821.

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model1) 

0.8635

## 3. Logistic Regression Classifier

In [26]:
from pyspark.ml.classification import LogisticRegression

classifier =LogisticRegression(featuresCol="features_normalized", labelCol="price_range")

model3 = classifier.fit(df2)

model3 = model3.transform(df2)

model3.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|price_range|            features| features_normalized|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|          1|[842.0,0.0,2.2,0....|[0.19111171637387...|[-4.6117991971757...|[3.25847702938626...|       2.0|
|          2|[1021.0,1.0,0.5,1...|[0.15067441929104...|[-4.5399931399192...|[7.52007943914062...|       2.0|
|          2|[563.0,1.0,0.5,1....|[0.08834975833281...|[-4.4826035856861...|[7.68929870609152...|       2.0|
|          2|[615.0,1.0,2.5,0....|[0.09343238685565...|[-5.2814555859558...|[2.07710523953717...|       2.0|
|          1|[1821.0,1.0,1.2,0...|[0.30881155881155...|[-1.3744837438914...|[0.01698546786883...|       1.0|
|          1|[1859.0,0.0,0.5,1...|[0.31984446509067...|[0.74947976026659...|[0.08969699656940...|       1.0|
|          3|[1821.

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("price_range")
    
binEval.evaluate(model3) 

0.7605

# Accuracy Comparision between ML models

### Decision Tree Classifier ==> 81.15 %
### Random Forest Classifier ==> 85.65 %
### Logistic Regression ==> 76.05 %

by examining above evaluations i have found Random forest best working on this dataset and has more accuracy

# Deep Learning 
I am doing it using pytorch framework

In [28]:
!pip install torch torchvision

  Using cached https://files.pythonhosted.org/packages/13/70/54e9fb010fe1547bc4774716f11ececb81ae5b306c05f090f4461ee13205/torch-1.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/61/51/aa2770a70f612ce9a2fc7da3a1a93f9ecf8746788256fed6b691f9b31ca9/torchvision-0.6.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ba/90/8a24e6220cfcf6a3a0162535d5b926e774117e384ff921908e07e4c92bda/Pillow-7.1.1-cp36-cp36m-manylinux1_x86_64.whl
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
Target directory /home/spark/shared/user-libs/python3.6/libfuturize already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/Pillow-7.1.1.dist-info already exists. Specify --upgrade to forc

### Again taking our initial dataset in Pandas dataframe and converting it to pytorch tensor

In [29]:
df_data_1

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0,0.5,1,3,0,22,0.7,164,1,...,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0,1.7,0,4,1,10,0.8,139,8,...,381,1018,3220,13,8,18,1,0,1,3
7,1954,0,0.5,1,0,0,24,0.8,187,4,...,512,1149,700,16,3,5,1,1,1,0
8,1445,1,0.5,0,0,0,53,0.7,174,7,...,386,836,1099,17,1,20,1,0,0,0
9,509,1,0.6,1,2,1,9,0.1,93,5,...,1137,1224,513,19,10,12,1,0,0,0


In [30]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [31]:
label_df = df_data_1['price_range']
df_train_new = df_data_1.drop(labels='price_range', axis=1)

train_data = torch.tensor(df_train_new.values)
print(train_data.shape)

train_label = torch.tensor(label_df.values)
train_label

train_label1 = F.one_hot(train_label, num_classes=4)

torch.Size([2000, 20])


In [32]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Linear(20,17)
        self.layer2 = nn.Linear(17,15)
        self.layer3 = nn.Linear(15,13)
        self.layer4 = nn.Linear(13,10)
        self.layer5 = nn.Linear(10,8)
        self.layer6 = nn.Linear(8,4)
    
    def forward(self,x):
        
        x = F.relu(self.layer1(x))
        x = F.dropout(x)
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.dropout(x)
        x = F.relu(self.layer4(x))
        x = F.relu(self.layer5(x)) 
        x = self.layer6(x)
        
        return x
print(train_data.shape)
print(train_label.shape)
print(train_label)

torch.Size([2000, 20])
torch.Size([2000])
tensor([1, 2, 2,  ..., 3, 0, 3])


In [33]:
model = Classifier()

optimizer = optim.Adam(model.parameters(), lr=0.005)

criterian = nn.CrossEntropyLoss()

train_data = train_data.float()
train_lable = train_label.float()

epoch = 10000
for i in range(epoch):
    
    optimizer.zero_grad()
    
    output = model(train_data.float())
    loss = criterian(output,train_label)
    loss.backward()
    optimizer.step()
    
    print('At ',i,'/100 epoch loss is: ',loss.item())

At  0 /100 epoch loss is:  8.767779350280762
At  1 /100 epoch loss is:  5.484195709228516
At  2 /100 epoch loss is:  4.144908905029297
At  3 /100 epoch loss is:  3.1126255989074707
At  4 /100 epoch loss is:  2.597700595855713
At  5 /100 epoch loss is:  2.250540256500244
At  6 /100 epoch loss is:  1.9684960842132568
At  7 /100 epoch loss is:  1.8065251111984253
At  8 /100 epoch loss is:  1.6604270935058594
At  9 /100 epoch loss is:  1.5682059526443481
At  10 /100 epoch loss is:  1.4945271015167236
At  11 /100 epoch loss is:  1.447304606437683
At  12 /100 epoch loss is:  1.4271869659423828
At  13 /100 epoch loss is:  1.418051838874817
At  14 /100 epoch loss is:  1.385413408279419
At  15 /100 epoch loss is:  1.3814194202423096
At  16 /100 epoch loss is:  1.3729850053787231
At  17 /100 epoch loss is:  1.375566005706787
At  18 /100 epoch loss is:  1.362054467201233
At  19 /100 epoch loss is:  1.3701282739639282
At  20 /100 epoch loss is:  1.359167456626892
At  21 /100 epoch loss is:  1.3609

At  180 /100 epoch loss is:  0.9194445013999939
At  181 /100 epoch loss is:  0.9329240322113037
At  182 /100 epoch loss is:  0.9353368878364563
At  183 /100 epoch loss is:  0.9190411567687988
At  184 /100 epoch loss is:  0.9320564866065979
At  185 /100 epoch loss is:  0.9397959113121033
At  186 /100 epoch loss is:  0.9350596070289612
At  187 /100 epoch loss is:  0.9300249814987183
At  188 /100 epoch loss is:  0.9275970458984375
At  189 /100 epoch loss is:  0.9177674055099487
At  190 /100 epoch loss is:  0.9415305256843567
At  191 /100 epoch loss is:  0.9131444692611694
At  192 /100 epoch loss is:  0.9211146831512451
At  193 /100 epoch loss is:  0.9171464443206787
At  194 /100 epoch loss is:  0.910377562046051
At  195 /100 epoch loss is:  0.905182421207428
At  196 /100 epoch loss is:  0.9241195917129517
At  197 /100 epoch loss is:  0.9054490923881531
At  198 /100 epoch loss is:  0.9187336564064026
At  199 /100 epoch loss is:  0.9087516665458679
At  200 /100 epoch loss is:  0.90743994712

At  362 /100 epoch loss is:  0.7815998196601868
At  363 /100 epoch loss is:  0.7863196134567261
At  364 /100 epoch loss is:  0.7743098139762878
At  365 /100 epoch loss is:  0.8081145286560059
At  366 /100 epoch loss is:  0.7718796730041504
At  367 /100 epoch loss is:  0.7763850092887878
At  368 /100 epoch loss is:  0.7901785969734192
At  369 /100 epoch loss is:  0.7600828409194946
At  370 /100 epoch loss is:  0.7721942067146301
At  371 /100 epoch loss is:  0.7667984366416931
At  372 /100 epoch loss is:  0.760758638381958
At  373 /100 epoch loss is:  0.7700400948524475
At  374 /100 epoch loss is:  0.7701669931411743
At  375 /100 epoch loss is:  0.7696515917778015
At  376 /100 epoch loss is:  0.7558673620223999
At  377 /100 epoch loss is:  0.7797437310218811
At  378 /100 epoch loss is:  0.7590242028236389
At  379 /100 epoch loss is:  0.7853897213935852
At  380 /100 epoch loss is:  0.7666328549385071
At  381 /100 epoch loss is:  0.7663819789886475
At  382 /100 epoch loss is:  0.7605540156

At  549 /100 epoch loss is:  0.6209319829940796
At  550 /100 epoch loss is:  0.602697491645813
At  551 /100 epoch loss is:  0.61661297082901
At  552 /100 epoch loss is:  0.6181685924530029
At  553 /100 epoch loss is:  0.614670991897583
At  554 /100 epoch loss is:  0.6074019074440002
At  555 /100 epoch loss is:  0.6175762414932251
At  556 /100 epoch loss is:  0.6127781867980957
At  557 /100 epoch loss is:  0.6463429927825928
At  558 /100 epoch loss is:  0.6155762672424316
At  559 /100 epoch loss is:  0.6189233064651489
At  560 /100 epoch loss is:  0.6075008511543274
At  561 /100 epoch loss is:  0.6307129263877869
At  562 /100 epoch loss is:  0.6177141666412354
At  563 /100 epoch loss is:  0.5980479717254639
At  564 /100 epoch loss is:  0.6012685298919678
At  565 /100 epoch loss is:  0.6141433119773865
At  566 /100 epoch loss is:  0.60199373960495
At  567 /100 epoch loss is:  0.6024884581565857
At  568 /100 epoch loss is:  0.6415778994560242
At  569 /100 epoch loss is:  0.601340293884277

At  724 /100 epoch loss is:  0.4873368442058563
At  725 /100 epoch loss is:  0.4795728921890259
At  726 /100 epoch loss is:  0.5001673102378845
At  727 /100 epoch loss is:  0.4766225218772888
At  728 /100 epoch loss is:  0.4895084798336029
At  729 /100 epoch loss is:  0.473394513130188
At  730 /100 epoch loss is:  0.48530474305152893
At  731 /100 epoch loss is:  0.48739737272262573
At  732 /100 epoch loss is:  0.4749639630317688
At  733 /100 epoch loss is:  0.4868903160095215
At  734 /100 epoch loss is:  0.4622914493083954
At  735 /100 epoch loss is:  0.46829554438591003
At  736 /100 epoch loss is:  0.45399773120880127
At  737 /100 epoch loss is:  0.4769652485847473
At  738 /100 epoch loss is:  0.46388787031173706
At  739 /100 epoch loss is:  0.45649561285972595
At  740 /100 epoch loss is:  0.4797351062297821
At  741 /100 epoch loss is:  0.47855088114738464
At  742 /100 epoch loss is:  0.44810813665390015
At  743 /100 epoch loss is:  0.48720115423202515
At  744 /100 epoch loss is:  0.4

...

At  9299 /100 epoch loss is:  0.1831624060869217
At  9300 /100 epoch loss is:  0.15821310877799988
At  9301 /100 epoch loss is:  0.1551615297794342
At  9302 /100 epoch loss is:  0.1655275970697403
At  9303 /100 epoch loss is:  0.1766299605369568
At  9304 /100 epoch loss is:  0.15544474124908447
At  9305 /100 epoch loss is:  0.17049390077590942
At  9306 /100 epoch loss is:  0.15023131668567657
At  9307 /100 epoch loss is:  0.15310482680797577
At  9308 /100 epoch loss is:  0.1715744584798813
At  9309 /100 epoch loss is:  0.1609160453081131
At  9310 /100 epoch loss is:  0.1496427357196808
At  9311 /100 epoch loss is:  0.17942163348197937
At  9312 /100 epoch loss is:  0.15369237959384918
At  9313 /100 epoch loss is:  0.1739722043275833
At  9314 /100 epoch loss is:  0.16473416984081268
At  9315 /100 epoch loss is:  0.16915905475616455
At  9316 /100 epoch loss is:  0.17522010207176208
At  9317 /100 epoch loss is:  0.17196528613567352
At  9318 /100 epoch loss is:  0.16807520389556885
At  9319

At  9469 /100 epoch loss is:  0.16876305639743805
At  9470 /100 epoch loss is:  0.18465805053710938
At  9471 /100 epoch loss is:  0.16242556273937225
At  9472 /100 epoch loss is:  0.17733508348464966
At  9473 /100 epoch loss is:  0.15979130566120148
At  9474 /100 epoch loss is:  0.1670113354921341
At  9475 /100 epoch loss is:  0.17135126888751984
At  9476 /100 epoch loss is:  0.1498507559299469
At  9477 /100 epoch loss is:  0.15003634989261627
At  9478 /100 epoch loss is:  0.187561497092247
At  9479 /100 epoch loss is:  0.15976303815841675
At  9480 /100 epoch loss is:  0.15662483870983124
At  9481 /100 epoch loss is:  0.15610335767269135
At  9482 /100 epoch loss is:  0.17532490193843842
At  9483 /100 epoch loss is:  0.16724233329296112
At  9484 /100 epoch loss is:  0.1552560180425644
At  9485 /100 epoch loss is:  0.16540555655956268
At  9486 /100 epoch loss is:  0.18932858109474182
At  9487 /100 epoch loss is:  0.2058641016483307
At  9488 /100 epoch loss is:  0.18496577441692352
At  94

At  9642 /100 epoch loss is:  0.16495749354362488
At  9643 /100 epoch loss is:  0.15786761045455933
At  9644 /100 epoch loss is:  0.16914565861225128
At  9645 /100 epoch loss is:  0.16151779890060425
At  9646 /100 epoch loss is:  0.16005750000476837
At  9647 /100 epoch loss is:  0.16962429881095886
At  9648 /100 epoch loss is:  0.1717512160539627
At  9649 /100 epoch loss is:  0.1589985340833664
At  9650 /100 epoch loss is:  0.14963799715042114
At  9651 /100 epoch loss is:  0.16813242435455322
At  9652 /100 epoch loss is:  0.17416125535964966
At  9653 /100 epoch loss is:  0.18210318684577942
At  9654 /100 epoch loss is:  0.17948393523693085
At  9655 /100 epoch loss is:  0.16999690234661102
At  9656 /100 epoch loss is:  0.16566520929336548
At  9657 /100 epoch loss is:  0.170747309923172
At  9658 /100 epoch loss is:  0.14821526408195496
At  9659 /100 epoch loss is:  0.17477349936962128
At  9660 /100 epoch loss is:  0.15807299315929413
At  9661 /100 epoch loss is:  0.16106866300106049
At  

At  9810 /100 epoch loss is:  0.18224453926086426
At  9811 /100 epoch loss is:  0.20685885846614838
At  9812 /100 epoch loss is:  0.17999885976314545
At  9813 /100 epoch loss is:  0.1711704134941101
At  9814 /100 epoch loss is:  0.17092300951480865
At  9815 /100 epoch loss is:  0.18925927579402924
At  9816 /100 epoch loss is:  0.18054305016994476
At  9817 /100 epoch loss is:  0.15719255805015564
At  9818 /100 epoch loss is:  0.1747794896364212
At  9819 /100 epoch loss is:  0.178758442401886
At  9820 /100 epoch loss is:  0.16173143684864044
At  9821 /100 epoch loss is:  0.18388481438159943
At  9822 /100 epoch loss is:  0.18582651019096375
At  9823 /100 epoch loss is:  0.15699972212314606
At  9824 /100 epoch loss is:  0.17846445739269257
At  9825 /100 epoch loss is:  0.17726005613803864
At  9826 /100 epoch loss is:  0.16275684535503387
At  9827 /100 epoch loss is:  0.16876474022865295
At  9828 /100 epoch loss is:  0.18391497433185577
At  9829 /100 epoch loss is:  0.15959154069423676
At  

At  9992 /100 epoch loss is:  0.1724957823753357
At  9993 /100 epoch loss is:  0.15121500194072723
At  9994 /100 epoch loss is:  0.15552622079849243
At  9995 /100 epoch loss is:  0.1564360111951828
At  9996 /100 epoch loss is:  0.1657053828239441
At  9997 /100 epoch loss is:  0.1463632434606552
At  9998 /100 epoch loss is:  0.1691063493490219
At  9999 /100 epoch loss is:  0.15422864258289337


In [36]:
output_check = (model(train_data))
output_check

tensor([[ -18.2097,    9.2705,   -0.3725,  -27.2269],
        [ -68.0196,  -17.2184,   -1.2256,   -3.7805],
        [ -55.5025,  -10.6432,   -0.2057,   -9.5235],
        ...,
        [-164.9006,  -50.5300,   -0.2529,   14.8606],
        [  34.3166,   27.5726,    1.1588,  -31.8794],
        [-122.6070,  -36.2014,   -2.8881,    0.7798]], grad_fn=<AddmmBackward>)

In [38]:
def accuracy(output_check,train_label):
    list1 = []
    for e in output_check:
        max_index = 0
        for i in range(len(e)):
            if e[i].item() > e[max_index].item():
                max_index = i
        list1.append(max_index)

    count = 0
    for x,y in zip(list1, train_label):
        if x!=y:
            count+=1
    return (len(list1)-count)/len(list1)

print('Accuracy: ', accuracy(output_check, train_label)*100)

Accuracy:  94.0
